In [2]:
import cx_Oracle
import pandas as pd
def Get_Validation(corrected_Part,Se_Supplier):

    # Database connection parameters
    dsn = cx_Oracle.makedsn("10.199.104.127", "1521", service_name="scrubbing")
    connection = cx_Oracle.connect(user="Read_Only", password="READ_ONLY", dsn=dsn)
    cursor = connection.cursor()

    # Create a string of MPNs for SQL IN clause
    input_str =', '.join([f"'{corrected_Part}'"])
    input_supplier= ', '.join([f"'{Se_Supplier}'"])
    #corrected_Part=str(corrected_Part)
    #Se_Supplier=str(Se_Supplier)

    # SQL query
    query = f"SELECT com_id, com_partnum , cm.get_man_name(man_id) as se_supplier FROM cm.xlp_se_component WHERE NAN_PARTNUM = NONALPHA({input_str}) and cm.get_man_name(man_id)={input_supplier}"
    cursor.execute(query)
    rows=cursor.fetchall()
    dict_r={}
    lst_r=[]
    if rows:
        for row in rows:
            dict_r['COM_ID']=row[0]
            dict_r['COM_PARTNUM']=row[1]
            dict_r['SE_SUPPLIER']=row[2]
            lst_r.append(dict_r)
            return pd.DataFrame(lst_r)
    else:
        return pd.DataFrame([{'COM_ID':'','COM_PARTNUM':'','SE_SUPPLIER':''}])
    cursor.close()
    connection.close()

In [7]:
Get_Validation('RC1206FR-075-K11A','Yageo')

,COM_ID,COM_PARTNUM,SE_SUPPLIER
0,1847335114,RC1206FR-075K11A,Yageo


### Validation COM_LUT_ACQ

In [46]:
import cx_Oracle

def Val_COM_ACQ_LUT(MPN, Supplier):
    # Database connection parameters
    dsn = cx_Oracle.makedsn("10.199.104.127", "1521", service_name="scrubbing")
    connection = cx_Oracle.connect(user="Read_Only", password="READ_ONLY", dsn=dsn)
    cursor = connection.cursor()

    # First Logic: Fetch rows where both partnumber and supplier match
    query = """
    SELECT COM_PARTNUM, cm.get_man_name(man_id) SE_SUPPLIER, com_id 
    FROM cm.xlp_se_component 
    WHERE NAN_PARTNUM = NONALPHA(:1) AND cm.get_man_name(man_id) = :2
    """
    cursor.execute(query, (MPN, Supplier))
    rows = cursor.fetchall()

    # Initialize a variable to check if the exact match was found
    exact_match_found = False

    if rows:
        # If rows are found, it means we have an exact match
        db_partnumber, db_supplier, db_com_id = rows[0]
        status = "EXACT"
        result = {
            "status": status,
            "COM_PARTNUM": db_partnumber,
            "SE_SUPPLIER": db_supplier,
            "COM_ID": db_com_id
        }
        exact_match_found = True

    if not exact_match_found:
        # Second Logic: Fetch last_com_id from alias_table
        alias_query = """
        SELECT last_com_id 
        FROM cm.tbl_part_acquisition 
        WHERE OLD_NAN_PARTNUM = NONALPHA(:1) AND cm.get_man_name(OLD_MAN_ID) = :2
        """
        cursor.execute(alias_query, (MPN, Supplier))
        alias_row = cursor.fetchone()

        if alias_row:
            last_com_id = alias_row[0]
            # Use last_com_id to get corresponding details from the first table
            fetch_com_query = """
            SELECT COM_PARTNUM, cm.get_man_name(man_id) SE_SUPPLIER, com_id 
            FROM cm.xlp_se_component 
            WHERE com_id = :1
            """
            cursor.execute(fetch_com_query, (last_com_id,))
            final_row = cursor.fetchone()
            if final_row:
                status = "ACQ"
                db_partnumber, db_supplier, db_com_id = final_row
                result = {
                    "status": status,
                    "COM_PARTNUM": db_partnumber,
                    "SE_SUPPLIER": db_supplier,
                    "COM_ID": db_com_id
                }
                exact_match_found = True
            else:
                exact_match_found = False  # No match found, continue to third logic
        else:
            exact_match_found = False  # No alias found, continue to third logic

    if not exact_match_found:
        # Third Logic: Fetch se_com_id from lut table
        lut_query = """
        SELECT se_com_id 
        FROM cm.part_lookup 
        WHERE NAN_INPUT_PART = NONALPHA(:1) AND cm.get_man_name(SE_MAN_ID) = :2
        """
        cursor.execute(lut_query, (MPN, Supplier))
        lut_row = cursor.fetchone()

        if lut_row:
            se_com_id = lut_row[0]
            
            # Use se_com_id to get corresponding details from the first table
            fetch_se_com_query = """
            SELECT COM_PARTNUM, cm.get_man_name(man_id) SE_SUPPLIER, com_id 
            FROM cm.xlp_se_component 
            WHERE com_id = :1
            """
            cursor.execute(fetch_se_com_query, (se_com_id,))
            final_se_row = cursor.fetchone()
            
            if final_se_row:
                status = "LUT"
                db_partnumber, db_supplier, db_com_id = final_se_row
                result = {
                    "status": status,
                    "COM_PARTNUM": db_partnumber,
                    "SE_SUPPLIER": db_supplier,
                    "COM_ID": db_com_id
                }
                exact_match_found = True
            else:
                # If no matching entry found for se_com_id
                status = "NOT MATCH"
                result = {
                    "status": status,
                    "COM_PARTNUM": '',
                    "SE_SUPPLIER": '',
                    "COM_ID": ''
                }
        else:
            # If no lut entry is found, proceed to handle this scenario
            status = "NOT MATCH"
            result = {
                    "status": status,
                    "COM_PARTNUM": '',
                    "SE_SUPPLIER": '',
                    "COM_ID": ''
                }

    # Close the database connection
    cursor.close()
    connection.close()
    # Return or print the result
    return result


In [47]:
Val_COM_ACQ_LUT('KOAX-WS-2XOR-WS','Turck')

{'status': 'EXACT',
 'COM_PARTNUM': 'KOAX-WS-2-XOR-WS',
 'SE_SUPPLIER': 'Turck',
 'COM_ID': 62966537}

In [26]:
Val_COM_ACQ_LUT('RC55-LFC294KDA','TT Electronics')

{'status': 'LUT',
 'COM_PARTNUM': 'RC55LFC2943DA',
 'SE_SUPPLIER': 'TT Electronics',
 'COM_ID': 411993558}

In [35]:
Val_COM_ACQ_LUT('104006A45M010','Molex')

62089911
('1300610130', 'Molex', 62089911)


{'status': 'ACQ',
 'COM_PARTNUM': '1300610130',
 'SE_SUPPLIER': 'Molex',
 'COM_ID': 62089911}

In [42]:
Val_COM_ACQ_LUT('','')

{'status': 'NOT MATCH', 'COM_PARTNUM': '', 'SE_SUPPLIER': '', 'COM_ID': ''}

In [38]:
Val_COM_ACQ_LUT('AM29DL323GB90WDI','Advanced Micro Devices, Inc.')

26734385
('Am29DL323GB90WDI', 'Infineon Technologies AG', 26734385)


{'status': 'ACQ',
 'COM_PARTNUM': 'Am29DL323GB90WDI',
 'SE_SUPPLIER': 'Infineon Technologies AG',
 'COM_ID': 26734385}